Importing libraries

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Scraping wikipedia page for Toronto table

In [2]:
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req = requests.get(wiki_link).text
soup = BeautifulSoup(req, 'lxml')

In [3]:
table = soup.find('table')
print(table)

Postcode = []
Borough = []
Neighbourhood = []

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [4]:
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)


In [5]:
unique_p = set(Postcode)

Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

Create dataframe from dictionary that includes table data from wikipedia

In [6]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
df_toronto.head(14)

import geocoder
from geopy.geocoders import Nominatim

Create nominatim var. Add "Canada" column to parse address in order to format address for geocode retrieval of coordinates 

In [7]:
nom = Nominatim(user_agent = None, timeout = 20)
df_toronto['Canada']= 'Canada'
df_toronto.head(10)

C:\Users\Camelia Oancea\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


,Borough,Neighbourhood,Postcode,Canada
0,Scarborough,Woburn,M1G,Canada
1,Downtown Toronto,Rosedale,M4W,Canada
2,Downtown Toronto,"Adelaide, King",M5H,Canada
3,York,Weston,M9N,Canada
4,Scarborough,"Agincourt North, Milliken",M1V,Canada
5,Scarborough,Birch Cliff,M1N,Canada
6,Etobicoke,Westmount,M9P,Canada
7,East Toronto,Riverdale,M4K,Canada
8,North York,Glencairn,M6B,Canada
9,Scarborough,Cedarbrae,M1H,Canada


Concatenate Neighborhood and add country column to format data for coordinates retrieval via geocode. Extract latitude and longitude data in the dataframe

In [8]:
df_toronto['Full Address']=df_toronto['Neighbourhood']+', '+df_toronto['Canada']
df_toronto.head(5)
df_toronto['Coordinates'] = df_toronto['Full Address'].apply(nom.geocode)
df_toronto['Latitude']=df_toronto['Coordinates'].apply(lambda x: x.latitude if x!=None else None)
df_toronto['Longitude'] = df_toronto['Coordinates'].apply(lambda x: x.longitude if x!=None else None)

In [9]:
df_toronto.head(10)

,Borough,Neighbourhood,Postcode,Canada,Full Address,Coordinates,Latitude,Longitude
0,Scarborough,Woburn,M1G,Canada,"Woburn, Canada","(Woburn, Scarborough, Toronto, Ontario, M1H 2A...",43.759824,-79.225291
1,Downtown Toronto,Rosedale,M4W,Canada,"Rosedale, Canada","(Rosedale, Calgary, Alberta, T2N 0M9, Canada, ...",51.064632,-114.082240
2,Downtown Toronto,"Adelaide, King",M5H,Canada,"Adelaide, King, Canada","(Terroni, 57, Adelaide Street East, King East,...",43.650938,-79.375294
3,York,Weston,M9N,Canada,"Weston, Canada","(Weston, York, Toronto, Ontario, M9N 2B3, Cana...",43.700161,-79.516247
4,Scarborough,"Agincourt North, Milliken",M1V,Canada,"Agincourt North, Milliken, Canada",None,NaN,NaN
5,Scarborough,Birch Cliff,M1N,Canada,"Birch Cliff, Canada","(Birch Cliff Drive, Salmon Cove, Newfoundland ...",47.776166,-53.175100
6,Etobicoke,Westmount,M9P,Canada,"Westmount, Canada","(Westmount, Agglomération de Montréal, Montréa...",45.485733,-73.596395
7,East Toronto,Riverdale,M4K,Canada,"Riverdale, Canada","(Riverdale, Old Toronto, Toronto, Ontario, M4M...",43.665470,-79.352594
8,North York,Glencairn,M6B,Canada,"Glencairn, Canada","(Glencairn, 1, Allen Road, Glen Park, North Yo...",43.708712,-79.440685
9,Scarborough,Cedarbrae,M1H,Canada,"Cedarbrae, Canada","(Cedarbrae, Calgary, Alberta, T2W 4M2, Canada,...",50.956318,-114.129323


Check NAs in the Coordinates retrieval via geocode

In [12]:
df_toronto[df_toronto['Coordinates'].isnull()]
df_toronto.isna().sum()

Borough           0
Neighbourhood     0
Postcode          0
Canada            0
Full Address      0
Coordinates      21
Latitude         21
Longitude        21
dtype: int64

Additional formatting of addresses to get data for missing Coordinates: splitting neighbourhood on comma, concatenate with country name and then call geocode to fill NAs 

In [13]:
df_toronto2 = df_toronto["Neighbourhood"].str.split(",", n = 1, expand = True)
df_toronto['First Address']=df_toronto2[0]
df_toronto['Final Address'] = df_toronto['First Address'] + ', '+df_toronto['Canada']
df_toronto['Coordinates2'] = df_toronto['Final Address'].apply(nom.geocode)
df_toronto['Lat']=df_toronto['Coordinates2'].apply(lambda x: x.latitude if x!=None else None)
df_toronto['Long'] = df_toronto['Coordinates2'].apply(lambda x: x.longitude if x!=None else None)

In [15]:
df_toronto.head(5)

,Borough,Neighbourhood,Postcode,Canada,Full Address,Coordinates,Latitude,Longitude,First Address,Final Address,Coordinates2,Lat,Long
0,Scarborough,Woburn,M1G,Canada,"Woburn, Canada","(Woburn, Scarborough, Toronto, Ontario, M1H 2A...",43.759824,-79.225291,Woburn,"Woburn, Canada","(Woburn, Scarborough, Toronto, Ontario, M1H 2A...",43.759824,-79.225291
1,Downtown Toronto,Rosedale,M4W,Canada,"Rosedale, Canada","(Rosedale, Calgary, Alberta, T2N 0M9, Canada, ...",51.064632,-114.082240,Rosedale,"Rosedale, Canada","(Rosedale, Calgary, Alberta, T2N 0M9, Canada, ...",51.064632,-114.082240
2,Downtown Toronto,"Adelaide, King",M5H,Canada,"Adelaide, King, Canada","(Terroni, 57, Adelaide Street East, King East,...",43.650938,-79.375294,Adelaide,"Adelaide, Canada","(Adelaide, Adelaide Metcalfe, Middlesex County...",43.004638,-81.721032
3,York,Weston,M9N,Canada,"Weston, Canada","(Weston, York, Toronto, Ontario, M9N 2B3, Cana...",43.700161,-79.516247,Weston,"Weston, Canada","(Weston, York, Toronto, Ontario, M9N 2B3, Cana...",43.700161,-79.516247
4,Scarborough,"Agincourt North, Milliken",M1V,Canada,"Agincourt North, Milliken, Canada",None,NaN,NaN,Agincourt North,"Agincourt North, Canada","(Agincourt North, Scarborough, Toronto, Ontari...",43.808038,-79.266439


Check again for any missing coordinates data after secound round of address formatting

In [16]:
df_toronto.isna().sum()

Borough           0
Neighbourhood     0
Postcode          0
Canada            0
Full Address      0
Coordinates      21
Latitude         21
Longitude        21
First Address     0
Final Address     0
Coordinates2      2
Lat               2
Long              2
dtype: int64

In [17]:
df_toronto[df_toronto['Coordinates2'].isnull()]

,Borough,Neighbourhood,Postcode,Canada,Full Address,Coordinates,Latitude,Longitude,First Address,Final Address,Coordinates2,Lat,Long
12,North York,CFB Toronto,M3K,Canada,"CFB Toronto, Canada",None,NaN,NaN,CFB Toronto,"CFB Toronto, Canada",None,NaN,NaN
73,Etobicoke,"Beaumond Heights, Jamestown, Mount Olive, Silv...",M9V,Canada,"Beaumond Heights, Jamestown, Mount Olive, Silv...",None,NaN,NaN,Beaumond Heights,"Beaumond Heights, Canada",None,NaN,NaN


In [19]:
df_toronto.iloc[73:74, :]

,Borough,Neighbourhood,Postcode,Canada,Full Address,Coordinates,Latitude,Longitude,First Address,Final Address,Coordinates2,Lat,Long
73,Etobicoke,"Beaumond Heights, Jamestown, Mount Olive, Silv...",M9V,Canada,"Beaumond Heights, Jamestown, Mount Olive, Silv...",None,NaN,NaN,Beaumond Heights,"Beaumond Heights, Canada",None,NaN,NaN


Manually extract coordinates data for the missing addresses (#2)

In [20]:
df_toronto.iloc[73,11] = nom.geocode('Etobicoke, Canada').latitude

In [21]:
df_toronto.iloc[73]

Borough                                                  Etobicoke
Neighbourhood    Beaumond Heights, Jamestown, Mount Olive, Silv...
Postcode                                                       M9V
Canada                                                      Canada
Full Address     Beaumond Heights, Jamestown, Mount Olive, Silv...
Coordinates                                                   None
Latitude                                                       NaN
Longitude                                                      NaN
First Address                                     Beaumond Heights
Final Address                             Beaumond Heights, Canada
Coordinates2                                                  None
Lat                                                        43.6436
Long                                                           NaN
Name: 73, dtype: object

In [22]:
df_toronto.iloc[73,12] = nom.geocode('Etobicoke, Canada').longitude

In [23]:
df_toronto.iloc[73,:]

Borough                                                  Etobicoke
Neighbourhood    Beaumond Heights, Jamestown, Mount Olive, Silv...
Postcode                                                       M9V
Canada                                                      Canada
Full Address     Beaumond Heights, Jamestown, Mount Olive, Silv...
Coordinates                                                   None
Latitude                                                       NaN
Longitude                                                      NaN
First Address                                     Beaumond Heights
Final Address                             Beaumond Heights, Canada
Coordinates2                                                  None
Lat                                                        43.6436
Long                                                      -79.5656
Name: 73, dtype: object

In [24]:
df_toronto.iloc[12:13,:]

,Borough,Neighbourhood,Postcode,Canada,Full Address,Coordinates,Latitude,Longitude,First Address,Final Address,Coordinates2,Lat,Long
12,North York,CFB Toronto,M3K,Canada,"CFB Toronto, Canada",None,NaN,NaN,CFB Toronto,"CFB Toronto, Canada",None,NaN,NaN


In [25]:
df_toronto.iloc[12,11] = nom.geocode('Canadian Forces Base, Toronto, Canada').latitude

In [26]:
df_toronto.iloc[12,:]

Borough                   North York
Neighbourhood            CFB Toronto
Postcode                         M3K
Canada                        Canada
Full Address     CFB Toronto, Canada
Coordinates                     None
Latitude                         NaN
Longitude                        NaN
First Address            CFB Toronto
Final Address    CFB Toronto, Canada
Coordinates2                    None
Lat                          43.7516
Long                             NaN
Name: 12, dtype: object

In [27]:
df_toronto.iloc[12, 12] = nom.geocode('Canadian Forces Base, Toronto, Canada').longitude

In [28]:
df_toronto.iloc[12,:]

Borough                   North York
Neighbourhood            CFB Toronto
Postcode                         M3K
Canada                        Canada
Full Address     CFB Toronto, Canada
Coordinates                     None
Latitude                         NaN
Longitude                        NaN
First Address            CFB Toronto
Final Address    CFB Toronto, Canada
Coordinates2                    None
Lat                          43.7516
Long                        -79.4712
Name: 12, dtype: object

Check again if any NAs still in dataframe

In [29]:
df_toronto.isna().sum()

Borough           0
Neighbourhood     0
Postcode          0
Canada            0
Full Address      0
Coordinates      21
Latitude         21
Longitude        21
First Address     0
Final Address     0
Coordinates2      2
Lat               0
Long              0
dtype: int64

In [30]:
df_toronto.columns

Index(['Borough', 'Neighbourhood', 'Postcode', 'Canada', 'Full Address',
       'Coordinates', 'Latitude', 'Longitude', 'First Address',
       'Final Address', 'Coordinates2', 'Lat', 'Long'],
      dtype='object')

In [31]:
df_toronto_fin = df_toronto.iloc[:,[2,0,1,11,12]]

Create the final dataset with the required columns / final dataframe used for clustering 

In [32]:
df_toronto_fin.head(10)

,Postcode,Borough,Neighbourhood,Lat,Long
0,M1G,Scarborough,Woburn,43.759824,-79.225291
1,M4W,Downtown Toronto,Rosedale,51.064632,-114.082240
2,M5H,Downtown Toronto,"Adelaide, King",43.004638,-81.721032
3,M9N,York,Weston,43.700161,-79.516247
4,M1V,Scarborough,"Agincourt North, Milliken",43.808038,-79.266439
5,M1N,Scarborough,Birch Cliff,47.776166,-53.175100
6,M9P,Etobicoke,Westmount,45.485733,-73.596395
7,M4K,East Toronto,Riverdale,43.665470,-79.352594
8,M6B,North York,Glencairn,43.708712,-79.440685
9,M1H,Scarborough,Cedarbrae,50.956318,-114.129323


In [33]:
df_toronto_fin = df_toronto_fin.rename(index = str, columns = {'Lat':'Latitude', 'Long': 'Longitude'})

In [34]:
df_toronto_fin.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1G,Scarborough,Woburn,43.759824,-79.225291
1,M4W,Downtown Toronto,Rosedale,51.064632,-114.082240
2,M5H,Downtown Toronto,"Adelaide, King",43.004638,-81.721032
3,M9N,York,Weston,43.700161,-79.516247
4,M1V,Scarborough,"Agincourt North, Milliken",43.808038,-79.266439
5,M1N,Scarborough,Birch Cliff,47.776166,-53.175100
6,M9P,Etobicoke,Westmount,45.485733,-73.596395
7,M4K,East Toronto,Riverdale,43.665470,-79.352594
8,M6B,North York,Glencairn,43.708712,-79.440685
9,M1H,Scarborough,Cedarbrae,50.956318,-114.129323


In [35]:
df_toronto_fin.shape

(84, 5)

Map Toronto

In [36]:
import folium
from folium import plugins

In [37]:
address = "Toronto, Canada"

In [39]:
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

C:\Users\Camelia Oancea\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto, Canada are 43.653963, -79.387207.


In [40]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

In [45]:
# add markers to map
for lat, lng, label in zip(df_toronto_fin['Latitude'], df_toronto_fin['Longitude'], df_toronto_fin['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

Get venues data from Foursquare and categories to cluster Toronto neighbourhoods based on number of venues for each category

In [46]:
CLIENT_ID = 'SNC4SZWPI4XGBXGW0QY1UGXWYBXRUQR2FC0URSN4KNWNQCR5' 
CLIENT_SECRET = 'M2QPYPAAYJY2MSM45PZAEMSSRFCAIWOXLCOLPT2E4YHJIMJI' 
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SNC4SZWPI4XGBXGW0QY1UGXWYBXRUQR2FC0URSN4KNWNQCR5
CLIENT_SECRET:M2QPYPAAYJY2MSM45PZAEMSSRFCAIWOXLCOLPT2E4YHJIMJI


In [47]:
limit = 100
radius = 500

In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
       
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
      
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
      
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
toronto_venues = getNearbyVenues(names=df_toronto_fin['Neighbourhood'],
                                   latitudes=df_toronto_fin['Latitude'],
                                   longitudes=df_toronto_fin['Longitude']
                                  )

Woburn
Rosedale
Adelaide, King
Weston
Agincourt North, Milliken
Birch Cliff
Westmount
Riverdale
Glencairn
Cedarbrae
Thorncliffe Park
East Toronto
CFB Toronto
Newtonbrook, Willowdale
Commerce Court
Flemingdon Park
Moore Park
Chinatown, Grange Park, Kensington Market
Dorset Park, Scarborough Town Centre, Wexford Heights
The Annex, Yorkville
CN Tower, King and Spadina, Railway Lands, South Niagara
First Canadian Place, Underground city
Ryerson
The Kingsway
Bedford Park
Rouge, Malvern
Berczy Park
Toronto Islands, Union Station
Deer Park, Rathnelly, South Hill
Parkwoods
The Beaches
Islington Avenue
Forest Hill North
Steeles West
Alderwood, Long Branch
Tam O'Shanter
St. James Town
Highland Creek, Rouge Hill, Port Union
Clairlea, Golden Mile, Oakridge
Emery, Humberlea
Victoria Village
Parkdale, Roncesvalles
Agincourt
Maryvale, Wexford
Kingsview Village
Henry Farm
Silver Hills, York Mills
Keelesdale, Mount Dennis, Silverthorn
Little Portugal, Trinity
Mimico NW, The Queensway West, South of Blo

In [53]:
print(toronto_venues.shape)
toronto_venues.head()

(1985, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Woburn,43.759824,-79.225291,GoodLife Fitness Scarborough Cedarbrae Mall,43.758303,-79.228533,Gym
1,Woburn,43.759824,-79.225291,Shoppers Drug Mart,43.760718,-79.224505,Pharmacy
2,Woburn,43.759824,-79.225291,Staples Cedarbrae,43.759851,-79.228335,Paper / Office Supplies Store
3,Woburn,43.759824,-79.225291,TD Canada Trust,43.761520,-79.226557,Bank
4,Woburn,43.759824,-79.225291,The Beer Store,43.759714,-79.225487,Beer Store


In [54]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,11,11,11,11,11,11
"Agincourt North, Milliken",31,31,31,31,31,31
"Alderwood, Long Branch",10,10,10,10,10,10
Bayview Village,12,12,12,12,12,12
"Beaumond Heights, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",10,10,10,10,10,10
Bedford Park,2,2,2,2,2,2
Berczy Park,100,100,100,100,100,100
CFB Toronto,10,10,10,10,10,10
"CN Tower, King and Spadina, Railway Lands, South Niagara",90,90,90,90,90,90


Cluster neighborhoods based on Venues and categories from Foursquare

In [55]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Amphitheater,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Create dummy dataframe for clustering

In [56]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Zoo,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Amphitheater,Aquarium,Arepa Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.090909,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,"Agincourt North, Milliken",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.032258,0.000000,0.032258,0.000000,0.00,0.000000,0.032258,0.000000,0.000000
2,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,"Beaumond Heights, Jamestown, Mount Olive, Silv...",0.000000,0.0,0.000000,0.000000,0.100000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,Bedford Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,Berczy Park,0.000000,0.0,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.020000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,CFB Toronto,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,"CN Tower, King and Spadina, Railway Lands, Sou...",0.000000,0.0,0.000000,0.000000,0.022222,0.000000,0.033333,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.011111,0.000000,0.000000,0.011111
9,"Cabbagetown, St. James Town",0.000000,0.0,0.000000,0.000000,0.018182,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [61]:
from sklearn.cluster import KMeans

In [62]:

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 3, 1, 1, 1, 1])

In [114]:
toronto_grouped["Cluster Labels"] = kmeans.labels_

In [115]:
toronto_grouped.shape

(77, 271)

In [116]:
toronto_grouped.columns

Index(['Neighborhood', 'Zoo', 'Accessories Store', 'Adult Boutique',
       'Afghan Restaurant', 'American Restaurant', 'Amphitheater', 'Aquarium',
       'Arepa Restaurant', 'Art Gallery',
       ...
       'Video Game Store', 'Video Store', 'Vietnamese Restaurant',
       'Warehouse Store', 'Whisky Bar', 'Wine Bar', 'Wings Joint',
       'Women's Store', 'Yoga Studio', 'Cluster Labels'],
      dtype='object', length=271)

In [118]:
toronto_datven = toronto_grouped[['Neighborhood','Cluster Labels']]

In [119]:
toronto_datven.head(5)

,Neighborhood,Cluster Labels
0,Agincourt,1
1,"Agincourt North, Milliken",1
2,"Alderwood, Long Branch",1
3,Bayview Village,1
4,"Beaumond Heights, Jamestown, Mount Olive, Silv...",1


In [121]:
df_toronto_fin = df_toronto_fin.rename(index=str, columns={"Neighbourhood": "Neighborhood"})
df_toronto_fin.columns

Index(['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'], dtype='object')

Merge dataframes to get cluster labels and latitude and longitude data for neighbourhoods to map 

In [128]:
df_toronto_fin = pd.merge(df_toronto_fin, toronto_datven, on = 'Neighborhood', how = 'right')

In [134]:
df_toronto_fin.head(5)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
0,M1G,Scarborough,Woburn,43.759824,-79.225291,1
1,M4W,Downtown Toronto,Rosedale,51.064632,-114.082240,1
2,M9N,York,Weston,43.700161,-79.516247,1
3,M1V,Scarborough,"Agincourt North, Milliken",43.808038,-79.266439,1
4,M9P,Etobicoke,Westmount,45.485733,-73.596395,1


In [136]:
import matplotlib.cm as cm
import matplotlib.colors as colors

Create map of clusters

In [137]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_toronto_fin['Latitude'], df_toronto_fin['Longitude'], df_toronto_fin['Neighborhood'], df_toronto_fin['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters